In [1]:
set.seed(30034)

In [2]:
# install.packages("corrplot")
# install.packages("plot.matrix")
# install.packages("ggplot2")
# install.packages("reshape2")
# install.packages("ramify")
# install.packages("SAVER")
# install.packages("MASS")

In [3]:
library(corrplot)
library(plot.matrix)
library(ggplot2)
library(reshape2)
library(ramify)
library(SAVER)
library(MASS)
library(stats)

corrplot 0.90 loaded


Attaching package: ‘ramify’


The following object is masked from ‘package:graphics’:

    clip




# Define constants

In [4]:
N <- 240
V <- 441
x1 <- 21
x2 <- 21
p <- 0
nsrcs <- 6

# Question 1: Synthetic dataset generation, data preporcessing, & data visualization

## 1 TC line plot

In [5]:
AV <- c(0,20,0,0,0,0)
IV <- c(30,45,60,40,40,40)
ones <- c(15,20,25,15,20,25)

In [6]:
tempTC = matrix(, nrow = N, ncol = nsrcs)
# Add elements to the empty matrix
for (i in rep(1:nsrcs)) {
    tempTC[, i] <- append(rep(0, AV[i]), rep(rep(1:0, times=c(ones[i], IV[i] - ones[i])), length.out=N - AV[i]))
}

In [7]:
TC <- scale(tempTC) 

In [8]:
pdf(file = "./images/TC_subplots-gen.pdf", width = 8, height = 8)
par(mfrow = c(3, 3))
for (i in rep(1:nsrcs)) {
    plot(TC[,i], type="S", main=paste("TC", i), xlab="", ylab="")
}
dev.off()

pdf 
  2

## 2 TC correlation plot

In [9]:
cc <- cor(TC)

In [10]:
pdf(file = "./images/TC_variable_correlations.pdf", width = 4, height = 4)
corrplot(cc, type="upper", method="color", title = "TC variables orrelation", mar=c(0,0,1,0))
dev.off()

pdf 
  2

## 3 tmpSM plot

In [11]:
ones_v = list(rep(2:6),rep(2:6), rep(8:13), rep(8:13), rep(15:19), rep(15:19))
ones_h = list(rep(2:6), rep(15:19), rep(2:6), rep(15:19), rep(2:6), rep(15:19))

In [12]:
tempSM <- array(rep(NA, nsrcs*x1*x2), dim = c(x1, x2, nsrcs))
for (i in rep(1:nsrcs)) {
    tempM <- matrix(0, nrow = x1, ncol = x2)
    vertical <- unlist(ones_v[i])
    horizontal <- unlist(ones_h[i])
    tempM[vertical, horizontal] <- matrix(1, nrow = length(vertical), ncol = length(horizontal))
    tempSM[,,i] <- tempM
}

In [13]:
SM <- array(rep(NA, nsrcs*x1*x2), dim = c(nsrcs, x1*x2))
for (i in rep(1:nsrcs)) {
    SM[i,] <- as.vector(tempSM[,,i])
}

In [14]:
SM_cor <- cor(t(SM))

In [15]:
pdf(file = "./images/SM_subplots-gen.pdf", width = 8, height = 5)
par(mar=c(2,2,2,4.5))
par(mfrow = c(3, 3))
for (i in rep(1:nsrcs)) {
    plot(tempSM[,,i], main=paste("SM", i),  xlab="", ylab="", col = heat.colors(8))
}
dev.off()

pdf 
  2

In [16]:
pdf(file = "./images/SM_variable_correlations.pdf", width = 4, height = 4)
corrplot(SM_cor, type="upper", method="color", title = "SM variables correlation", mar=c(0,0,1,0))
dev.off()

pdf 
  2

## 4 Gaussian noise

In [17]:
t_var <- 0.25
s_var <- 0.015

In [18]:
gamma_t <- matrix(rnorm(N*nsrcs, mean = 0, sd = sqrt(t_var)), nrow = N, ncol = nsrcs)
gamma_s <- matrix(rnorm(nsrcs*V, mean = 0, sd = sqrt(s_var)), nrow = nsrcs, ncol = V)

In [19]:
gamma_t_cor <- cor(gamma_t)
gamma_s_cor <- cor(t(gamma_s))

In [20]:
pdf(file = "./images/gamma_t_corr.pdf", width = 4, height = 4)
corrplot(gamma_t_cor, type="upper", method="color", title = expression(Gamma[t] ~ variables ~ correlation), mar=c(0,0,1,0))
dev.off()

pdf 
  2

In [21]:
pdf(file = "./images/gamma_s_corr.pdf", width = 4, height = 4)
corrplot(gamma_s_cor, type="upper", method="color", title = expression(Gamma[s] ~ variables ~ correlation), mar=c(0,0,1,0))
dev.off()

pdf 
  2

In [22]:
pdf(file = "./images/gamma_distributions.pdf", width = 7, height = 5)
par(mfrow = c(1, 2))
hist(as.vector(gamma_t), breaks=100, main = expression(Gamma[t] ~ distribution), xlab = "")
abline(v=0, col="blue", lwd=3)
abline(v=-1.96*sqrt(t_var), col="red", lwd=2)
text(-1.96*sqrt(t_var), 60, expression(-1.96*sigma))
abline(v=1.96*sqrt(t_var), col="red", lwd=2)
text(1.96*sqrt(t_var), 60, expression(1.96*sigma))
hist(as.vector(gamma_s), breaks=100, main = expression(Gamma[s] ~ distribution), xlab="")
abline(v=0, col="blue", lwd=2)
abline(v=-1.96*sqrt(s_var), col="red", lwd=2)
text(-1.96*sqrt(s_var), 100, expression(-1.96*sigma))
abline(v=1.96*sqrt(s_var), col="red", lwd=2)
text(1.96*sqrt(s_var), 100, expression(1.96*sigma))
dev.off()

pdf 
  2

In [23]:
pdf(file = "./images/gammas_subset_corr.pdf", width = 4, height = 4)
corrplot(cor(gamma_t%*%gamma_s)[1:20, 1:20], type="upper", method="color", title = expression(Subset~of~Gamma[t]*Gamma[s]~CM), mar=c(0,0,1,0))
dev.off()

pdf 
  2

## 5 Sythetic dataset

In [24]:
tempX <- (TC+gamma_t)%*%(SM+gamma_s)

In [25]:
partial_X <- melt(tempX[, sample(c(1:240), size=100)])

In [26]:
pdf("./images/X_partial.pdf" , width = 7, height = 4)
p <- ggplot(data=partial_X, aes(x=Var1, y=value, color=as.factor(Var2))) + geom_line() + theme(legend.position = "none") +  labs(title = "Randomly selected time-series from X", x= "time",  y= "values")
print(p)
dev.off()

pdf 
  2

In [27]:
pdf(file = "./images/X_var.pdf", width = 6, height = 6)
plot(diag(var(tempX)), main="Variance of 441 variables", xlab="Variables", ylab="Variance")
dev.off()

pdf 
  2

In [28]:
X <- scale(tempX)

# Question 2: Data analysis, results visualization, & performance metrics


## 1 Retriving SM and TC for RR

In [29]:
D = TC

In [30]:
A_LSR <- abs(solve(t(D)%*%D)%*%t(D)%*%X)
D_LSR <- X%*%t(A_LSR)

In [31]:
pdf(file = "./images/retrieved_SM_TC_subplots.pdf", width = 8, height = 5)
par(mar=c(2,1,1,4.5))
par(mfrow = c(3, 4))
for (i in 1:nsrcs) {
    plot(matrix(A_LSR[i, ], nrow = 21, ncol = 21), main=paste("Retrieved SM", i),  xlab="", ylab="", col = heat.colors(8))
    plot(D_LSR[,i], type="S", main=paste("Retrieved TC", i), xlab="", ylab="")
}
dev.off()

pdf 
  2

In [32]:
pdf(file = "./images/col3_col30.pdf", width = 6, height = 6)
plot(x=D_LSR[,3], y=X[,30], main = "Retrieved TC column 3 vs X column 30", xlab = "Retrieved TC", ylab="X")
dev.off()

pdf 
  2

In [33]:
pdf(file = "./images/col4_col30.pdf", width = 6, height = 6)
plot(x=D_LSR[,4], y=X[,30], main = "Retrieved TC column 4 vs X column 30", xlab = "Retrieved TC", ylab="X")
dev.off()

pdf 
  2

In [34]:
length(D_LSR[,4])

[1] 240

## 2 Estimate PR parameters

In [35]:
generate_A_RR <- function(lambda) {
    return(abs(solve(t(D)%*%D+lambda*V*diag(6))%*%t(D)%*%X))
}

In [36]:
cor_RR <- function(lambda) {
    D_RR <- X%*%t(generate_A_RR(lambda))
    return(sum(calc.maxcor(TC, D_RR)))
}

In [37]:
optimize(cor_RR, interval=c(0, 1))

$minimum
[1] 0.9999339

$objective
[1] 5.271137

In [38]:
# Pick lambda=1

In [39]:
(c_TLSR <- sum(calc.maxcor(TC, D_LSR)))

[1] 5.260546

In [40]:
(c_TRR.1 <- cor_RR(1))

[1] 5.271136

In [41]:
pdf(file="./images/A_RR_first_vec.pdf", width = 6, height = 6)
par(mar=c(2,1,1,4.5))

plot(matrix(generate_A_RR(1000)[1, ], nrow = 21, ncol = 21), main=expression(First~vector~of~A[RR]~with~lambda==1000),  xlab="", ylab="",  breaks = c(0,1), col = heat.colors(8))
# plot(generate_A_RR(1000)[1, ], ylim=c(-1e-3, 1e-3), main=expression(First~vector~of~A[RR]~with~lambda==1000), ylab="")
dev.off()

pdf 
  2

In [42]:
pdf(file="./images/A_LSR_first_vec.pdf", width = 6, height = 6)
par(mar=c(2,1,1,4.5))
plot(matrix(A_LSR[1,], nrow = 21, ncol = 21), main=expression(First~vector~of~A[LSR]),  xlab="", ylab="", breaks = c(0,1), col = heat.colors(8))

# plot(A_LSR[1,], ylim=c(-1.5, 1.5), main=expression(First~vector~of~A[LSR]), ylab="")
dev.off()

pdf 
  2

## 3 Retriving SM and TC for LR

In [43]:
rhos <- seq(0, 1, 0.05)

In [44]:
generate_A_LR <- function(X, TC, rho) {
    step <- 1/(norm(TC%*%t(TC)) * 1.1)
    thr <- rho*N*step
    Ao <- matrix(0, nsrcs, 1)
    A <- matrix(0, nsrcs, 1)
    A_LR <- matrix(0, nsrcs, x1*x2)

    for (k in 1:(x1*x2)) {
        A <- Ao+step*(t(TC) %*% (X[,k]-(TC%*%Ao)))
        A <- (1/(1+thr)) * (sign(A)*pmax(replicate(nsrcs, 0), abs(A)-thr))

        for (i in 1:10) {
            Ao <- A
            A <- Ao+step * (t(TC)%*%(X[,k]-(TC%*%Ao)))
            A <- (1/(1+thr)) * (sign(A)*pmax(replicate(nsrcs, 0), abs(A)-thr))
        }
        A_LR[,k] <- A
    }
    return(abs(A_LR))
}

In [45]:
generate_X <- function() {
    gamma_t <- matrix(rnorm(N*nsrcs, mean = 0, sd = sqrt(t_var)), nrow = N, ncol = nsrcs)
    gamma_s <- matrix(rnorm(nsrcs*V, mean = 0, sd = sqrt(s_var)), nrow = nsrcs, ncol = V)
    return(scale((TC+gamma_t)%*%(SM+gamma_s)))
}

In [46]:
MSEs <- matrix(0, length(rhos), 1)
for (i in 1:length(rhos)) {
    MSE = matrix(0, 10, 1)
    for (j in 1:10) {
        newX <- generate_X()
        tempA_LR = generate_A_LR(newX, TC, rhos[i])
        tempD_LR = newX%*%t(tempA_LR)
        temp = newX - tempD_LR%*%tempA_LR 
        MSE[j] <- sum(temp^2) / (N*V)
    }
    MSEs[i] <- mean(MSE)
}

In [47]:
pdf(file="./images/LR_MSE.pdf", width = 6, height = 6)
plot(x =rhos ,y=as.vector(MSEs), type="l", main = "MSE of LR parameters", ylab = "MSE", xlab = expression(rho))
dev.off()

pdf 
  2

In [48]:
pdf(file="./images/LR_MSE_zoom.pdf", width = 6, height = 6)
plot(x =rhos[10:21] ,y=as.vector(MSEs)[10:21], type="l", main = "Zoomed MSE of LR parameters", ylab = "MSE", xlab = expression(rho))
dev.off()

pdf 
  2

## 4 Estimate LR parameters

In [49]:
rho = 0.6
lambda = 1

In [50]:
A_RR <- generate_A_RR(lambda)
D_RR <- X%*%t(A_RR)
A_LR <- generate_A_LR(X, TC, rho)
D_LR <- X%*%t(A_LR)

In [51]:
# TC and D_RR
(c_TRR <- sum(calc.maxcor(TC, D_RR)))

[1] 5.271136

In [52]:
# SM and A_RR
(c_SRR <- sum(calc.maxcor(SM, A_RR)))

[1] 356.1611

In [53]:
# TC and  D_LR
(c_TLR <- sum(calc.maxcor(TC, D_LR)))

[1] 5.394234

In [54]:
# SM and A_LR
(c_SLR <- sum(calc.maxcor(SM, A_LR)))

[1] 362.7012

In [55]:
pdf(file = "./images/SM_TC_RR_LR_subplots.pdf", width = 8, height = 10)
par(mar=c(2,1,1,4.5))
par(mfrow = c(6, 4))
for (i in 1:nsrcs) {
    plot(matrix(A_RR[i, ], nrow = 21, ncol = 21), main=paste("Retrieved SM RR", i),  xlab="", ylab="", col = heat.colors(8))
    plot(D_RR[,i], type="S", main=paste("Retrieved TC RR", i), xlab="", ylab="")
    plot(matrix(A_LR[i, ], nrow = 21, ncol = 21), main=paste("Retrieved SM LR", i),  xlab="", ylab="", col = heat.colors(8))
    plot(D_LR[,i], type="S", main=paste("Retrieved TC LR", i), xlab="", ylab="")
}
dev.off()

pdf 
  2

## 5 Estimate PCs of TCs

In [56]:
TC_prccomp <- prcomp(TC)

In [57]:
pdf(file = "./images/eigenvalue.pdf", width = 6, height = 6)
barplot(TC_prccomp$sdev^2, main = "Eigenvalues of PCs", xlab="Principle component", ylab="Eigenvalue", names=1:6)
dev.off()

pdf 
  2

In [58]:
Z <- TC_prccomp$x

In [59]:
pdf(file = "./images/PCA_TC_subplots.pdf", width = 8, height = 5)
par(mfrow = c(3, 4))
par(mar=c(2, 2, 2, 2))
for (i in rep(1:nsrcs)) {
    plot(Z[,i], type="S", main=paste("Z variable", i), xlab="", ylab="")    
    plot(TC[,i], type="S", main=paste("TC variable", i), xlab="", ylab="")
}
dev.off()

pdf 
  2

In [60]:
A_PCR <- generate_A_LR(X, Z, 0.001)
D_PCR <- X%*%t(A_PCR)

In [61]:
pdf(file = "./images/D_PCR_A_PCR_subplots.pdf", width = 8, height = 5)
par(mar=c(2,1,1,4.5))
par(mfrow = c(3, 4))
for (i in 1:nsrcs) {
    plot(matrix(A_PCR[i, ], nrow = 21, ncol = 21), main=paste("Retrieved SM PCR", i),  xlab="", ylab="", col = heat.colors(8))
    plot(D_PCR[,i], type="S", main=paste("Retrieved TC PCR", i), xlab="", ylab="")
}
dev.off()

pdf 
  2